In [ ]:
# Imports
import os
import json
import sqlite3
from typing import Optional

import requests
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [ ]:
# Load environment variables
load_dotenv(override=True)
openai_api_key = os.getenv("OPENAI_API_KEY")
openrouter_api_key = os.getenv("OPENROUTER_API_KEY")
# EPL data: TheSportsDB (free key 123, or set THE_SPORTS_DB_API_KEY)
thesportsdb_api_key = os.getenv("THE_SPORTS_DB_API_KEY", "123")

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
if openrouter_api_key:
    print(f"OpenRouter API key exists and begins {openrouter_api_key[:7]}")
else:
    print("OpenRouter API key not set")
print(f"TheSportsDB API: enabled (EPL league 4328, key {'custom' if thesportsdb_api_key != '123' else '123 (free)'})")

# Clients: OpenAI (GPT) and OpenRouter (Claude)
openai_client = OpenAI(api_key=openai_api_key) if openai_api_key else None
openrouter_url = "https://openrouter.ai/api/v1"
openrouter_client = (
    OpenAI(api_key=openrouter_api_key, base_url=openrouter_url)
    if openrouter_api_key
    else None
)

GPT_MODEL = "gpt-4.1-mini"
CLAUDE_MODEL = "anthropic/claude-3.5-sonnet"

In [ ]:
# System prompt: EPL assistant
system_message = """
You are an AI assistant for English Premier League (EPL) fans.
Help users with: current standings, next fixtures for any team, player stats, and ticket booking.
When the user asks for standings, fixtures, player info, latest match score, or who scored in a match (goalscorers), use the available tools to fetch real data.
For ticket booking:
- When the user asks to book a ticket (e.g. "book ticket for this match"), you MUST first ask for their name or email if they have not already provided it. Only then call book_ticket with both match_description (short text like 'Brentford vs Arsenal 2026-02-12 20:00') and user_identifier (the name or email they gave).
- Never call book_ticket without a user_identifier; the tool will reject the call. Always get the user's name or email before booking.
Booking is educational only: it saves to a local database; there is no real ticket provider.
Answer concisely. If a tool fails, explain politely. Respond in markdown when useful.
"""

In [ ]:
# EPL data: TheSportsDB API (https://www.thesportsdb.com/league/4328-english-premier-league)
TSDB_BASE = "https://www.thesportsdb.com/api/v1/json"
EPL_LEAGUE_ID = 4328  # English Premier League


def _tsdb_request(script: str, params: Optional[dict] = None) -> str:
    """GET TheSportsDB v1 API; key in URL. Returns JSON string or error."""
    url = f"{TSDB_BASE}/{thesportsdb_api_key}/{script}"
    try:
        r = requests.get(url, params=params or {}, timeout=10)
        r.raise_for_status()
        return json.dumps(r.json() if r.text else {})
    except requests.RequestException as e:
        return f"Error: TheSportsDB request failed: {e}"
    except Exception as e:
        return f"Error: {e}"


def _tsdb_standings() -> str:
    """Premier League table (lookuptable.php?l=4328)."""
    return _tsdb_request("lookuptable.php", params={"l": EPL_LEAGUE_ID})


def _tsdb_search_team(team_name: str) -> Optional[str]:
    """Get team ID by name (searchteams.php). Prefer EPL (idLeague 4328)."""
    q = team_name.strip().replace(" ", "_")
    if not q:
        return None
    raw = _tsdb_request("searchteams.php", params={"t": q})
    if raw.startswith("Error:"):
        return None
    try:
        data = json.loads(raw)
        teams = data.get("teams") or []
        for t in teams:
            if str(t.get("idLeague")) == str(EPL_LEAGUE_ID):
                return t.get("idTeam")
        return teams[0].get("idTeam") if teams else None
    except Exception:
        return None


def _tsdb_team_fixtures(team_name: str) -> str:
    """Next EPL events for team: league-wide then filter; if none, team-specific (eventsnext)."""
    raw = _tsdb_request("eventsnextleague.php", params={"id": EPL_LEAGUE_ID})
    if raw.startswith("Error:"):
        return raw
    try:
        data = json.loads(raw)
        events = data.get("events") or []
        team_lower = team_name.strip().lower()
        out = []
        for e in events:
            home = (e.get("strHomeTeam") or "").strip()
            away = (e.get("strAwayTeam") or "").strip()
            if team_lower in home.lower() or team_lower in away.lower():
                date_event = e.get("dateEvent") or ""
                time_event = (e.get("strTime") or "")[:5]
                out.append(f"{home} vs {away} {date_event} {time_event}")
        if out:
            return json.dumps(out[:15])
        # Fallback: get next event for this team specifically (free API returns 1 league event)
        team_id = _tsdb_search_team(team_name)
        if not team_id:
            return json.dumps(["No upcoming matches found for that team."])
        raw2 = _tsdb_request("eventsnext.php", params={"id": team_id})
        if raw2.startswith("Error:"):
            return json.dumps(["No upcoming matches found for that team."])
        data2 = json.loads(raw2)
        events2 = data2.get("events") or data2.get("results") or []
        for e in events2:
            home = (e.get("strHomeTeam") or "").strip()
            away = (e.get("strAwayTeam") or "").strip()
            date_event = e.get("dateEvent") or ""
            time_event = (e.get("strTime") or "")[:5]
            out.append(f"{home} vs {away} {date_event} {time_event}")
        return json.dumps(out[:15] if out else ["No upcoming matches found for that team."])
    except Exception as e:
        return f"Error: {e}"


def _tsdb_team_last_result(team_name: str) -> str:
    """Last match result for a team (eventslast.php) with score."""
    team_id = _tsdb_search_team(team_name)
    if not team_id:
        return "Error: Team not found. Try full name (e.g. Manchester City, Arsenal)."
    raw = _tsdb_request("eventslast.php", params={"id": team_id})
    if raw.startswith("Error:"):
        return raw
    try:
        data = json.loads(raw)
        results = data.get("results") or data.get("events") or []
        if not results:
            return json.dumps(["No recent match result found for that team."])
        e = results[0]
        home = e.get("strHomeTeam") or ""
        away = e.get("strAwayTeam") or ""
        hs = e.get("intHomeScore")
        aws = e.get("intAwayScore")
        date_event = e.get("dateEvent") or ""
        score = f"{hs}-{aws}" if hs is not None and aws is not None else "-"
        return json.dumps([{"strEvent": e.get("strEvent"), "dateEvent": date_event, "intHomeScore": hs, "intAwayScore": aws, "score": f"{home} {score} {away}"}])
    except Exception as e:
        return f"Error: {e}"


def _tsdb_match_goalscorers(team_name: str) -> str:
    """Goalscorers (and assists) for the team's most recent match (lookuptimeline.php)."""
    team_id = _tsdb_search_team(team_name)
    if not team_id:
        return "Error: Team not found. Try full name (e.g. Manchester City, Arsenal)."
    raw = _tsdb_request("eventslast.php", params={"id": team_id})
    if raw.startswith("Error:"):
        return raw
    try:
        data = json.loads(raw)
        results = data.get("results") or data.get("events") or []
        if not results:
            return json.dumps(["No recent match found for that team."])
        event_id = results[0].get("idEvent")
        if not event_id:
            return json.dumps(["No event id for last match."])
    except Exception as e:
        return f"Error: {e}"
    raw2 = _tsdb_request("lookuptimeline.php", params={"id": event_id})
    if raw2.startswith("Error:"):
        return raw2
    try:
        data2 = json.loads(raw2)
        timeline = data2.get("timeline") or []
        goals = []
        for t in timeline:
            if (t.get("strTimeline") or "").strip().lower() == "goal":
                goals.append({
                    "strPlayer": t.get("strPlayer"),
                    "intTime": t.get("intTime"),
                    "strAssist": t.get("strAssist") or "",
                    "strTeam": t.get("strTeam"),
                })
        return json.dumps(goals if goals else ["No goal timeline data for this match."])
    except Exception as e:
        return f"Error: {e}"


def _tsdb_player_search(player_name: str) -> str:
    """Search player by name (searchplayers.php?p=Name_With_Underscores)."""
    q = player_name.strip().replace(" ", "_")
    if not q:
        return "Error: player_name is required."
    raw = _tsdb_request("searchplayers.php", params={"p": q})
    if raw.startswith("Error:"):
        return raw
    try:
        data = json.loads(raw)
        players = data.get("player") or []
        # Prefer Soccer / EPL; return first match or first result
        needle = player_name.strip().lower()
        for p in players:
            if p.get("strSport") == "Soccer" and needle in (p.get("strPlayer") or "").lower():
                return json.dumps([{"strPlayer": p.get("strPlayer"), "strTeam": p.get("strTeam"), "strPosition": p.get("strPosition"), "strNationality": p.get("strNationality"), "dateBorn": p.get("dateBorn")}])
        if players:
            p = players[0]
            return json.dumps([{"strPlayer": p.get("strPlayer"), "strTeam": p.get("strTeam"), "strPosition": p.get("strPosition"), "strNationality": p.get("strNationality"), "dateBorn": p.get("dateBorn")}])
        return json.dumps(["No player found matching that name."])
    except Exception as e:
        return f"Error: {e}"

In [ ]:
# SQLite DB for bookings (same folder as notebook / cwd)
DB_PATH = os.path.join(os.getcwd(), "epl_bookings.db")


def _init_db() -> None:
    """Create bookings table if it does not exist."""
    with sqlite3.connect(DB_PATH) as conn:
        conn.execute(
            """CREATE TABLE IF NOT EXISTS bookings (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                match_description TEXT NOT NULL,
                user_identifier TEXT,
                booked_at TEXT NOT NULL
            )"""
        )


_init_db()


def get_standings() -> str:
    """Get current Premier League table (TheSportsDB)."""
    return _tsdb_standings()


def get_team_fixtures(team_name: str) -> str:
    """Get next fixtures for a team (e.g. Arsenal, Chelsea)."""
    if not team_name or not team_name.strip():
        return "Error: team_name is required."
    return _tsdb_team_fixtures(team_name)


def get_player_stats(player_name: str) -> str:
    """Get player info by name (team, position, nationality, DOB) from TheSportsDB."""
    if not player_name or not player_name.strip():
        return "Error: player_name is required."
    return _tsdb_player_search(player_name)


def get_last_match_result(team_name: str) -> str:
    """Get the latest match result (score) for a team."""
    if not team_name or not team_name.strip():
        return "Error: team_name is required."
    return _tsdb_team_last_result(team_name)


def get_match_goalscorers(team_name: str) -> str:
    """Get goalscorers (and assists) for the team's most recent match."""
    if not team_name or not team_name.strip():
        return "Error: team_name is required."
    return _tsdb_match_goalscorers(team_name)


def book_ticket(match_description: str, user_identifier: Optional[str] = None) -> str:
    """Save a ticket booking to local DB (educational only). Requires name/email. Returns confirmation or error."""
    if not match_description or not match_description.strip():
        return "Error: match_description is required."
    uid = (user_identifier or "").strip()
    if not uid:
        return "Error: Name or email is required for the booking. Please ask the user for their name or email, then call book_ticket again with user_identifier."
    # Normalize match_description: single line, max 300 chars (avoid JSON or huge paste)
    match_clean = " ".join(match_description.strip().split())[:300]
    from datetime import datetime
    try:
        with sqlite3.connect(DB_PATH) as conn:
            conn.execute(
                "INSERT INTO bookings (match_description, user_identifier, booked_at) VALUES (?, ?, ?)",
                (match_clean, uid, datetime.datetime.now(datetime.UTC).isoformat() + "Z"),
            )
        return f"Booking saved for {uid}: {match_clean}"
    except Exception as e:
        return f"Error saving booking: {e}"

In [ ]:
# Tool definitions for OpenAI/OpenRouter (function calling)
TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "get_standings",
            "description": "Get the current Premier League table (position, team, played, wins, draws, losses, goal difference, points).",
            "parameters": {"type": "object", "properties": {}, "required": []},
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_team_fixtures",
            "description": "Get the next Premier League fixtures for a given team (e.g. Arsenal, Chelsea).",
            "parameters": {
                "type": "object",
                "properties": {
                    "team_name": {"type": "string", "description": "Team name, e.g. Arsenal, Chelsea, Liverpool"},
                },
                "required": ["team_name"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_last_match_result",
            "description": "Get the latest match result (score) for a Premier League team (e.g. Manchester City, Arsenal). Use when user asks for latest score, last match result, or recent result.",
            "parameters": {
                "type": "object",
                "properties": {
                    "team_name": {"type": "string", "description": "Team name, e.g. Manchester City, Man City, Arsenal"},
                },
                "required": ["team_name"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_match_goalscorers",
            "description": "Get who scored (goalscorers and assists) in the team's most recent match. Use when user asks who scored, goalscorers, or scorers for a team's last fixture.",
            "parameters": {
                "type": "object",
                "properties": {
                    "team_name": {"type": "string", "description": "Team name, e.g. Manchester City, Arsenal"},
                },
                "required": ["team_name"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_player_stats",
            "description": "Get player info (name, team, position, nationality, date of birth) for a Premier League player by name (e.g. Salah, Haaland). Uses TheSportsDB search.",
            "parameters": {
                "type": "object",
                "properties": {
                    "player_name": {"type": "string", "description": "Player name or common name"},
                },
                "required": ["player_name"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "book_ticket",
            "description": "Book a ticket for a match (educational only: saves to local DB). You MUST ask the user for their name or email before calling if they have not provided it. Pass match_description as short human-readable text (e.g. 'Brentford vs Arsenal 2026-02-12 20:00') and user_identifier (name or email). Both are required.",
            "parameters": {
                "type": "object",
                "properties": {
                    "match_description": {"type": "string", "description": "Short match description, e.g. 'Brentford vs Arsenal 2026-02-12 20:00' (one line, no JSON)"},
                    "user_identifier": {"type": "string", "description": "User's name or email for the booking (required; ask the user if not provided)"},
                },
                "required": ["match_description", "user_identifier"],
            },
        },
    },
]

In [ ]:
# Handle tool calls from API message; return list of tool response dicts
def handle_tool_calls(message) -> list:
    """Execute tools and return list of {role: 'tool', content, tool_call_id}."""
    tool_calls = message.get("tool_calls") if isinstance(message, dict) else getattr(message, "tool_calls", None)
    if not tool_calls:
        return []
    responses = []
    for tc in tool_calls:
        if isinstance(tc, dict):
            name = tc["function"]["name"]
            arguments = json.loads(tc["function"].get("arguments", "{}"))
            tool_call_id = tc["id"]
        else:
            name = tc.function.name
            arguments = json.loads(tc.function.arguments or "{}")
            tool_call_id = tc.id
        content = "Unknown tool"
        if name == "get_standings":
            content = get_standings()
        elif name == "get_team_fixtures":
            content = get_team_fixtures(arguments.get("team_name", ""))
        elif name == "get_last_match_result":
            content = get_last_match_result(arguments.get("team_name", ""))
        elif name == "get_match_goalscorers":
            content = get_match_goalscorers(arguments.get("team_name", ""))
        elif name == "get_player_stats":
            content = get_player_stats(arguments.get("player_name", ""))
        elif name == "book_ticket":
            content = book_ticket(
                arguments.get("match_description", ""),
                arguments.get("user_identifier"),
            )
        # Console: which tool was called (and short result preview)
        preview = content[:100] + ("..." if len(content) > 100 else "")
        print(f"[EPL Tool] Called: {name}({arguments})", flush=True)
        print(f"[EPL Tool] Result: {preview}", flush=True)
        responses.append({"role": "tool", "content": content, "tool_call_id": tool_call_id})
    return responses

In [ ]:
# Chat: model choice, tools, streaming when no tool_calls
def get_client_and_model(model_choice: str):
    """Return (client, model_id) for the chosen UI option. Raises if key missing."""
    if model_choice == "Claude":
        if openrouter_client is None:
            raise ValueError("OpenRouter API key not set. Add OPENROUTER_API_KEY to .env")
        return openrouter_client, CLAUDE_MODEL
    if openai_client is None:
        raise ValueError("OpenAI API key not set. Add OPENAI_API_KEY to .env")
    return openai_client, GPT_MODEL


def _append_user_message(message: str, history: list) -> list:
    if not message or not message.strip():
        return history
    return history + [{"role": "user", "content": message.strip()}]


def transcribe_audio(audio_path) -> Optional[str]:
    """Transcribe audio file with OpenAI Whisper. Returns text or None if no API key / error."""
    if not audio_path or not openai_client:
        return None
    try:
        with open(audio_path, "rb") as f:
            r = openai_client.audio.transcriptions.create(model="whisper-1", file=f)
        return (r.text or "").strip() if getattr(r, "text", None) else None
    except Exception as e:
        print(f"[Whisper] Error: {e}", flush=True)
        return None


def _stream_reply(history: list, model_choice: str):
    """Stream when model returns content; handle tool_calls in a loop (no stream for tool round). Yields only chatbot state (list of {role, content})."""
    try:
        client, model_id = get_client_and_model(model_choice)
    except ValueError as e:
        yield history + [{"role": "assistant", "content": str(e)}]
        return
    messages = [{"role": "system", "content": system_message}] + list(history)
    reply = {"role": "assistant", "content": ""}

    stream = client.chat.completions.create(
        model=model_id,
        messages=messages,
        tools=TOOLS,
        stream=True,
    )
    tool_calls_accum = {}

    for chunk in stream:
        if not chunk.choices:
            continue
        delta = chunk.choices[0].delta
        finish_reason = chunk.choices[0].finish_reason

        if getattr(delta, "content", None):
            reply["content"] += delta.content
            yield history + [reply]
        if getattr(delta, "tool_calls", None):
            for tc in delta.tool_calls:
                i = tc.index
                if i not in tool_calls_accum:
                    tool_calls_accum[i] = {"id": "", "name": "", "arguments": ""}
                tool_calls_accum[i]["id"] += tc.id or ""
                tool_calls_accum[i]["name"] += tc.function.name or ""
                tool_calls_accum[i]["arguments"] += tc.function.arguments or ""

        if finish_reason == "tool_calls":
            assistant_msg = {
                "role": "assistant",
                "content": reply["content"] or "",
                "tool_calls": [
                    {
                        "id": tool_calls_accum[i]["id"],
                        "type": "function",
                        "function": {
                            "name": tool_calls_accum[i]["name"],
                            "arguments": tool_calls_accum[i]["arguments"],
                        },
                    }
                    for i in sorted(tool_calls_accum.keys())
                ],
            }
            messages.append(assistant_msg)
            messages.extend(handle_tool_calls(assistant_msg))
            while True:
                response = client.chat.completions.create(
                    model=model_id,
                    messages=messages,
                    tools=TOOLS,
                    stream=False,
                )
                msg = response.choices[0].message
                if msg.tool_calls:
                    msg_dict = {"role": "assistant", "content": msg.content or "", "tool_calls": msg.tool_calls}
                    messages.append(msg_dict)
                    messages.extend(handle_tool_calls(msg_dict))
                    continue
                reply["content"] = msg.content or ""
                yield history + [reply]
                return
        elif finish_reason == "stop":
            yield history + [reply]
            return

    if reply["content"]:
        yield history + [reply]
        return
    yield history + [reply]

In [ ]:
# Gradio UI: model selector, chat, submit / clear
with gr.Blocks(title="AI Assistant for EPL Fans", theme=gr.themes.Soft()) as demo:
    gr.Markdown("## AI Assistant for English Premier League (EPL) Fans")
    with gr.Row():
        model_dropdown = gr.Dropdown(
            choices=["GPT", "Claude"],
            value="GPT",
            label="Model",
            scale=0,
        )
    chatbot = gr.Chatbot(type="messages", label="Chat", height=400)
    msg = gr.Textbox(
        placeholder="Ask about standings, next match for a team, player stats, or book a ticket...",
        show_label=False,
        container=False,
    )
    with gr.Row():
        submit_btn = gr.Button("Send", variant="primary")
        clear_btn = gr.ClearButton([msg, chatbot], value="Clear chat")
    voice_audio = gr.Audio(
        sources=["microphone"],
        type="filepath",
        label="Or use voice (record then click Send voice)",
    )
    voice_btn = gr.Button("Send voice", variant="secondary")

    def on_submit(message, history):
        new_history = _append_user_message(message, history or [])
        return "", new_history

    def on_voice(audio, history):
        path = audio if isinstance(audio, str) else (audio.get("name") or audio.get("path") if isinstance(audio, dict) else None)
        if not path:
            return history or [], gr.update()
        if not openai_client:
            err = [{"role": "assistant", "content": "Voice input uses OpenAI Whisper. Set OPENAI_API_KEY in .env."}]
            return (history or []) + err, None
        text = transcribe_audio(path)
        if not text:
            return history or [], gr.update()
        new_history = _append_user_message(text, history or [])
        return new_history, None

    msg.submit(on_submit, [msg, chatbot], [msg, chatbot], queue=False).then(
        _stream_reply, [chatbot, model_dropdown], [chatbot], queue=True
    )
    submit_btn.click(on_submit, [msg, chatbot], [msg, chatbot], queue=False).then(
        _stream_reply, [chatbot, model_dropdown], [chatbot], queue=True
    )
    voice_btn.click(
        on_voice, [voice_audio, chatbot], [chatbot, voice_audio], queue=False
    ).then(_stream_reply, [chatbot, model_dropdown], [chatbot], queue=True)

demo.launch()